In [1]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd


##############  Variables ########################
PCA_NoF=10


# Read data from Excel files, US. MRI, Pix2Pix2D, CycleGAN3D, CycleGAN2D, AutoEncoder3D, UNet3D
Output = pd.read_excel('ADD OUTCOME FILE')
Input_Train = pd.read_excel('ADD TRAING DATASET')
Input_Val = pd.read_excel('ADD VALIDATION DATASET')
Input_Test= pd.read_excel('ADD TESTING DATASET')
#print(Output)
######Size datasets ################
trnnum_rows = len(Input_Train)
valnum_rows = len(Input_Val)
tstnum_rows = len(Input_Test)
#print("Shape of Train dataset:", Input_Train.shape)
#print("Shape of Validation dataset:", Input_Val.shape)
#print("Shape of Test dataset:", Input_Test.shape)
#print(Input_Train)
#print(Input_Val)
#print(Input_Test)


####Split Patient ID ################
PItrn =Input_Train.iloc[:, :1]
PIval =Input_Val.iloc[:, :1] 
PItst =Input_Test.iloc[:, :1] 
#print("Shape of IDtrn dataset:", PItrn.shape)
#print("Shape of IDval dataset:", PIval.shape)
#print("Shape of IDtst dataset:", PItst.shape)
#print(PItrn)
#print(PIval)
#print(PItst)


########### Concatenate data vertically #############
Total_data = pd.concat([Input_Train, Input_Val, Input_Test], ignore_index=True)
#print("Shape of Totaldataset:", Total_data.shape)
#print(Total_data)

############3 It's a good practice to standardize the data before PCA #####################3
scaler = StandardScaler()
X_scaled = scaler.fit_transform(Total_data.iloc[:, 3:])

# Create PCA instance: PCA for 50 components
pca = PCA(n_components=PCA_NoF)

# Fit on data
pca.fit(X_scaled)

# Transform the data
X_pca = pca.transform(X_scaled)
#print("Shape of PCA total dataset:", X_pca.shape)
#print(X_pca)
# Extract PCA results for Train, Validation, and Test sets


PCAtrn = pd.DataFrame(X_pca[:trnnum_rows, :])  # Rows from 0 to trnnum_rows-1 for training
PCAval = pd.DataFrame(X_pca[trnnum_rows:trnnum_rows+valnum_rows, :])  # Rows from trnnum_rows to trnnum_rows+valnum_rows-1 for validation
PCAtst = pd.DataFrame(X_pca[trnnum_rows+valnum_rows:, :])  # Rows from trnnum_rows+valnum_rows to the end for testing
#print("Shape of PCA Train dataset:", PCAtrain.shape)
#print("Shape of PCA Validation dataset:", PCAval.shape)
#print("Shape of PCA Test dataset:", PCAtst.shape)
#print(PCAtrain)
#print(PCAval)
#print(PCAtst)



#################Concatinate Patiant ID with PCA train, PCA val and PCA test ###############3

concatenated_trn = pd.concat([PItrn, PCAtrn], axis=1)
concatenated_val = pd.concat([PIval, PCAval], axis=1)
concatenated_tst = pd.concat([PItst, PCAtst], axis=1)
#print("Shape of concatinated trndataset:", concatenated_trn.shape)
#print("Shape of concatinated valdataset:", concatenated_val.shape)
#print("Shape of concatinated tstdataset:", concatenated_tst.shape)
#print(concatenated_trn)
#print(concatenated_val)
#print(concatenated_tst)


################Select related labels##########################33
###########For training #####################
numeric_label = {'low': 0, 'high': 1}
for patient in Output['PatientID']:
         for train_patient in concatenated_trn['PatientID']:
    
            if train_patient[:4] == patient[-4:]:
                grade = Output.loc[Output['PatientID'] == patient, 'Label'].squeeze()
                #print(grade)
                concatenated_trn.loc[concatenated_trn['PatientID'] == train_patient, 'Label'] = grade
                concatenated_trn.loc[concatenated_trn['PatientID'] == train_patient, 'Label_Num'] = int(numeric_label[grade])
#print(concatenated_trn)
#print("Shape of concatinated trndataset:", concatenated_trn.shape)
#print(concatenated_trn)

########### For Val #####################
for val_patient in concatenated_val['PatientID']:
        for patient in Output['PatientID']:
            if val_patient[:4] == patient[-4:]:
                grade = Output.loc[Output['PatientID'] == patient, 'Label'].squeeze()
                concatenated_val.loc[concatenated_val['PatientID'] == val_patient, 'Label'] = grade
                concatenated_val.loc[concatenated_val['PatientID'] == val_patient, 'Label_Num'] = int(numeric_label[grade])
#print("Shape of concatinated valdataset:", concatenated_val.shape)

########### For Test#####################
for tst_patient in concatenated_tst['PatientID']:
        for patient in Output['PatientID']:
            if tst_patient[:4] == patient[-4:]:
                grade = Output.loc[Output['PatientID'] == patient, 'Label'].squeeze()
                concatenated_tst.loc[concatenated_tst['PatientID'] == tst_patient, 'Label'] = grade
                concatenated_tst.loc[concatenated_tst['PatientID'] == tst_patient, 'Label_Num'] = int(numeric_label[grade])
#print("Shape of concatinaed tstdataset:", concatenated_tst.shape)
#print(concatenated_tst)

############Clean NaN################
clean_Trn = concatenated_trn.dropna(subset='Label_Num')
clean_Val = concatenated_val.dropna(subset='Label_Num')
clean_Tst= concatenated_tst.dropna(subset='Label_Num')

#print("Shape of concatinated tstdataset:", clean_Trn.shape)
#print("Shape of concatinated tstdataset:", clean_Val.shape)
#print("Shape of concatinated tstdataset:", clean_Tst.shape)
#print(clean_Trn)
#print(clean_Val)
#print(clean_Tst)


######################Classifiers##############################
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# Train the classifier
##### Split feature for predictor ###################3
X_trn=clean_Trn.iloc[:, 1:-2]
Y_trn=clean_Trn.iloc[:, -1:]
X_val=clean_Val.iloc[:, 1:-2]
Y_val=clean_Val.iloc[:, -1:]
X_tst=clean_Tst.iloc[:, 1:-2]
Y_tst=clean_Tst.iloc[:, -1:]

Total_X_trn = pd.concat([X_trn, X_val], ignore_index=True)#### Concatinate X train and val
Total_Y_trn = pd.concat([Y_trn, Y_val], ignore_index=True)#### Concatinate X train and val


In [231]:
#Random Forest

In [ ]:
#######################
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


#print("Shape of concatinated tstdataset:", Total_X_trn.shape)
#print("Shape of concatinated tstdataset:", Total_Y_trn.shape)
#print(Total_X_trn)
#print(Total_Y_trn)
#print("Shape of concatinated tstdataset:", X_tst.shape)
#print("Shape of concatinated tstdataset:", Y_tst.shape)
#print(X_tst)
#print(Y_tst)
#print(concatenated_tst.iloc[:, -1:])
# Initialize MLPClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd

# Train a Random Forest classifier
#clf = RandomForestClassifier(n_estimators=50000, random_state=42)
clf = RandomForestClassifier(n_estimators=100,  # Number of trees in the forest
                             criterion='gini',  # Splitting criterion (default is 'gini')
                             max_depth=100,   # Maximum depth of the trees (default is None)
                             min_samples_split=2,  # Minimum number of samples required to split an internal node
                             min_samples_leaf=1,   # Minimum number of samples required to be at a leaf node
                             max_features='auto',  # Number of features to consider for the best split
                             bootstrap=True,  # Whether bootstrap samples are used when building trees
                             random_state=11)  # Seed for random number generation
clf.fit(Total_X_trn, Total_Y_trn)

# Get predicted probabilities for the positive class
y_probs = clf.predict_proba(X_tst)[:, 1]  # Use [:, 1] for positive class probabilities
RFYpre = (y_probs >= 0.5).astype(int)  # Convert probabilities to binary predictions
#print([Ypre,Y_tst])
    
# Calculate accuracy
accuracyRF = accuracy_score(Y_tst, RFYpre)
print("Accuracy:", round(accuracyRF, 2))
#print(Y_tst)
#print(RFYpre)
# Calculate AUC
aucRF = roc_auc_score(Y_tst, RFYpre)
print("AUC:", round(aucRF, 2))

#class_counts = Total_Y_trn['Label_Num'].value_counts()
#print(class_counts)

#class_counts = Y_tst['Label_Num'].value_counts()
#print(class_counts)'


In [ ]:

print("RandF")
print("Accuracy:", round(accuracyRF, 2))
print("AUC:", round(aucRF, 2))
print("-----------------------")
